# Collaborative filtering on the MovieLense Dataset

###### This notebook is based on part of Chapter 9 of [BigQuery: The Definitive Guide](https://www.oreilly.com/library/view/google-bigquery-the/9781492044451/ "http://shop.oreilly.com/product/0636920207399.do") by Lakshmanan and Tigani.
### MovieLens dataset
To illustrate recommender systems in action, let’s use the MovieLens dataset. This is a dataset of movie reviews released by GroupLens, a research lab in the Department of Computer Science and Engineering at the University of Minnesota, through funding by the US National Science Foundation.

Download the data and load it as a BigQuery table using:

In [1]:
import os
PROJECT = "your-project-here" # REPLACE WITH YOUR PROJECT ID
BUCKET = "your-bucket-here" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "1.14"

In [2]:
%%bash
rm -r bqml_data
mkdir bqml_data
cd bqml_data
curl -O 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
unzip ml-20m.zip
bq --location=US mk --dataset \
    --description 'Movie Recommendations' \
    $PROJECT:movielens
bq --location=US load --source_format=CSV \
    --autodetect movielens.ratings ml-20m/ratings.csv
bq --location=US load  --source_format=CSV \
    --autodetect movielens.movies_raw ml-20m/movies.csv

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
Dataset 'cloud-training-demos:movielens' successfully created.






rm: cannot remove 'bqml_data': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189M  100  189M    0     0  47.8M      0  0:00:03  0:00:03 --:--:-- 47.8M
Waiting on bqjob_r2e948e79eaae8f1a_0000016ce4771a7e_1 ... (1s) Current status: DONE    

## Exploring the data
Two tables should now be available in <a href="https://console.cloud.google.com/bigquery">BigQuery</a>.

Collaborative filtering provides a way to generate product recommendations for users, or user targeting for products. The starting point is a table, <b>movielens.ratings</b>, with three columns: a user id, an item id, and the rating that the user gave the product. This table can be sparse -- users don’t have to rate all products. Then, based on just the ratings, the technique finds similar users and similar products and determines the rating that a user would give an unseen product. Then, we can recommend the products with the highest predicted ratings to users, or target products at users with the highest predicted ratings.

In [3]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.ratings
LIMIT 10

,userId,movieId,rating,timestamp
0,70139,3701,2.0,1047507862
1,70151,377,2.0,1009322468
2,70213,15,2.0,975531508
3,70232,5480,2.0,1347740273
4,70238,688,2.0,966975041
5,70243,4889,2.0,1116311911
6,70243,31221,2.0,1111820107
7,70246,454,2.0,834774425
8,70279,69844,2.0,1260993212
9,70285,54001,2.0,1415475292


A quick exploratory query yields that the dataset consists of over 138 thousand users, nearly 27 thousand movies, and a little more than 20 million ratings, confirming that the data has been loaded successfully.

In [4]:
%%bigquery --project $PROJECT
SELECT 
    COUNT(DISTINCT userId) numUsers,
    COUNT(DISTINCT movieId) numMovies,
    COUNT(*) totalRatings
FROM movielens.ratings

,numUsers,numMovies,totalRatings
0,138493,26744,20000263


On examining the first few movies using the query following query, we can see that the genres column is a formatted string:

In [5]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.movies_raw
WHERE movieId < 5

,movieId,title,genres
0,3,Grumpier Old Men (1995),Comedy|Romance
1,4,Waiting to Exhale (1995),Comedy|Drama|Romance
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


We can parse the genres into an array and rewrite the table as follows:

In [6]:
%%bigquery --project $PROJECT
CREATE OR REPLACE TABLE movielens.movies AS
    SELECT * REPLACE(SPLIT(genres, "|") AS genres)
    FROM movielens.movies_raw

""


In [7]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.movies
WHERE movieId < 5

,movieId,title,genres
0,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
1,3,Grumpier Old Men (1995),"[Comedy, Romance]"
2,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"


## Matrix factorization
Matrix factorization is a collaborative filtering technique that relies on factorizing the ratings matrix into two vectors called the user factors and the item factors. The user factors is a low-dimensional representation of a user_id and the item factors similarly represents an item_id.

We can create the recommender model using (<b>Optional</b>, takes 30 minutes. Note: we have a model we already trained if you want to skip this step):


In [ ]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', rating_col='rating')
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

Executing query with job ID: 3b814912-7050-47db-8bf2-ada580e81a12
Query executing: 825.49s

In [ ]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender`)

Note that we create a model as usual, except that the model_type is matrix_factorization and that we have to identify which columns play what roles in the collaborative filtering setup.

What did you get? Our model took an hour to train, and the training loss starts out extremely bad and gets driven down to near-zero over next the four iterations:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>4</td>
    <td>0.5734</td>
    <td>172.4057</td>
    <td>180.99</td>
  </tr>
  <tr>
    <td>3</td>
    <td>0.5826</td>
    <td>187.2103</td>
    <td>1,040.06</td>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6531</td>
    <td>4,758.2944</td>
    <td>219.46</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9776</td>
    <td>6,297.2573</td>
    <td>1,093.76</td>
  </tr>
  <tr>
    <td>0</td>
    <td>63,287,833,220.5795</td>
    <td>168,995,333.0464</td>
    <td>1,091.21</td>
  </tr>
</table>

However, the evaluation data loss is quite high, and much higher than the training data loss. This indicates that overfitting is happening, and so we need to add some regularization. Let’s do that next. Note the added l2_reg=0.2 (<b>Optional</b>, takes 30 minutes):

In [ ]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender_l2
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', 
        rating_col='rating', l2_reg=0.2)
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

In [12]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_l2`)

,training_run,iteration,loss,eval_loss,duration_ms
0,0,1,0.634286,0.736270,806193
1,0,0,85.287208,544.115891,874578


Now, we get faster convergence (three iterations instead of five), and a lot less overfitting. Here are our results:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6509</td>
    <td>1.4596</td>
    <td>198.17</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9829</td>
    <td>33,814.3017</td>
    <td>1,066.06</td>
  </tr>
  <tr>
    <td>0</td>
    <td>481,434,346,060.7928</td>
    <td>2,156,993,687.7928</td>
    <td>1,024.59</td>
  </tr>
</table>

By default, BigQuery sets the number of factors to be the log2 of the number of rows. In our case, since we have 20 million rows in the table, the number of factors would have been chosen to be 24. As with the number of clusters in K-Means clustering, this is a reasonable default but it is often worth experimenting with a number about 50% higher (36) and a number that is about a third lower (16):

In [13]:
%%bigquery --project $PROJECT
CREATE OR REPLACE MODEL movielens.recommender_16
options(model_type='matrix_factorization',
        user_col='userId', item_col='movieId', 
        rating_col='rating', l2_reg=0.2, num_factors=16)
AS

SELECT 
userId, movieId, rating
FROM movielens.ratings

""


In [14]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model:
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_16`)

,training_run,iteration,loss,eval_loss,duration_ms
0,0,2,0.599447,0.734480,504456
1,0,1,0.663859,0.736160,613588
2,0,0,52.284254,44.228623,990746


When we did that, we discovered that the evaluation loss was lower (0.97) with num_factors=16 than with num_factors=36 (1.67) or num_factors=24 (1.45). We could continue experimenting, but we are likely to see diminishing returns with further experimentation. So, let’s pick this as the final matrix factorization model and move on.

## Making recommendations

With the trained model, we can now provide recommendations. For example, let’s find the best comedy movies to recommend to the user whose userId is 903. In the query below, we are calling ML.PREDICT passing in the trained recommendation model and providing a set of movieId and userId to carry out the predictions on. In this case, it’s just one userId (903), but all movies whose genre includes Comedy.

In [15]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g
  WHERE g = 'Comedy'
))
ORDER BY predicted_rating DESC
LIMIT 5

,predicted_rating,movieId,title,userId
0,6.171499,117885,Grumpy Cat's Worst Christmas Ever (2014),903
1,6.084283,118888,Dave Chappelle: For What it's Worth (2004),903
2,5.987384,67812,Cheeky (Trasgredire) (2000),903
3,5.907633,72035,Dimensions of Dialogue (Moznosti dialogu) (1982),903
4,5.865696,26141,"Loved One, The (1965)",903


## Filtering out already rated movies
Of course, this includes movies the user has already seen and rated in the past. Let’s remove them:


In [16]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH seen AS (
    SELECT ARRAY_AGG(movieId) AS movies 
    FROM movielens.ratings
    WHERE userId = 903
  )
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g, seen
  WHERE g = 'Comedy' AND movieId NOT IN UNNEST(seen.movies)
))
ORDER BY predicted_rating DESC
LIMIT 5

,predicted_rating,movieId,title,userId
0,6.171499,117885,Grumpy Cat's Worst Christmas Ever (2014),903
1,6.084283,118888,Dave Chappelle: For What it's Worth (2004),903
2,5.987384,67812,Cheeky (Trasgredire) (2000),903
3,5.907633,72035,Dimensions of Dialogue (Moznosti dialogu) (1982),903
4,5.865696,26141,"Loved One, The (1965)",903


For this user, this happens to yield the same set of movies -- the top predicted ratings didn’t include any of the movies the user has already seen.

## Customer targeting

In the previous section, we looked at how to identify the top-rated movies for a specific user. Sometimes, we have a product and have to find the customers who are likely to appreciate it. Suppose, for example, we wish to get more reviews for movieId=96481 which has only one rating and we wish to send coupons to the 5 users who are likely to rate it the highest. We can identify those users using:

In [17]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH allUsers AS (
     SELECT DISTINCT userId
     FROM movielens.ratings
  )
  SELECT 
    96481 AS movieId, 
    (SELECT title FROM movielens.movies WHERE movieId=96481) title,
    userId
  FROM
    allUsers
))
ORDER BY predicted_rating DESC
LIMIT 5

,predicted_rating,movieId,title,userId
0,6.492631,96481,American Mullet (2001),122674
1,6.400817,96481,American Mullet (2001),129268
2,6.354193,96481,American Mullet (2001),58305
3,6.170367,96481,American Mullet (2001),67836
4,6.151552,96481,American Mullet (2001),14951


### Batch predictions for all users and movies
What if we wish to carry out predictions for every user and movie combination? Instead of having to pull distinct users and movies as in the previous query, a convenience function is provided to carry out batch predictions for all movieId and userId encountered during training. A limit is applied here, otherwise, all user-movie predictions will be returned and will crash the notebook.

In [18]:
%%bigquery --project $PROJECT
SELECT *
FROM ML.RECOMMEND(MODEL `cloud-training-demos.movielens.recommender_16`)
LIMIT 10

,predicted_rating,userId,movieId
0,3.612169,10496,4352
1,2.761672,10496,71168
2,3.536013,10496,81152
3,4.626323,10496,26368
4,3.100962,10496,38656
5,4.147963,10496,55040
6,3.226497,10496,4097
7,3.636138,10496,3586
8,3.554277,10496,55298
9,3.805262,10496,7171


As seen in a section above, it is possible to filter out movies the user has already seen and rated in the past. The reason already seen movies aren’t filtered out by default is that there are situations (think of restaurant recommendations, for example) where it is perfectly expected that we would need to recommend restaurants the user has liked in the past.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.